# Исследование надежности заемщиков


Задача: используя Python и библиотеку pandas провести исследование надёжности заёмщиков. Определить риск появления задолженности у различных категорий заёмщиков



## Откроем таблицу и изучим общую информацию о данных

**Импорт библиотеки pandas. Путь к файлу: `/datasets/data.csv`**

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Вывод первых 20 строк датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывед основной информации о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Вывод на экран количества пропущенных значений для каждого столбца**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения `days_employed` и `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для дальнейшего исследования.

**Выведите перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру. Проверим остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

#### Определим существует ли зависимость между количеством детей и возвратом кредита в срок?

### Исследуем данные

In [ ]:
#Ранее было установленно, что уникальные значения столбца children принимают значения от 0 до 5
#Опредилим (используя цикл for и сокращенную логическую индексацию) процентное соотношение между заёмщиками имевшими задолженность и всеми заёмщиками, имеющими одинаковое число детей
for i in range(0,6):
    relation = (data[(data['children'] == i) & (data['debt'] == 1)]['children'].count() / data[data['children'] == i]['children'].count())
    print(f'Доля заёмщиков с задолженностью среди всех заёмщиков с количеством детей: {i} составляет {relation:.1%}')  

Доля заёмщиков с задолженностью среди всех заёмщиков с количеством детей: 0 составляет 7.5%
Доля заёмщиков с задолженностью среди всех заёмщиков с количеством детей: 1 составляет 9.2%
Доля заёмщиков с задолженностью среди всех заёмщиков с количеством детей: 2 составляет 9.5%
Доля заёмщиков с задолженностью среди всех заёмщиков с количеством детей: 3 составляет 8.2%
Доля заёмщиков с задолженностью среди всех заёмщиков с количеством детей: 4 составляет 9.8%
Доля заёмщиков с задолженностью среди всех заёмщиков с количеством детей: 5 составляет 0.0%


In [23]:
# создадим сводную таблицу по количеству детей у заёмщиков 
data_pivot_children = data.pivot_table(index=['children'], values=['debt'], aggfunc = ['mean', 'count', 'sum']) 
data_pivot_children.columns = ['вероятность просрочки','количество клиентов в группе','количество просрочек в группе']
display(data_pivot_children)

,вероятность просрочки,количество клиентов в группе,количество просрочек в группе
children,,,
0,0.075438,14091,1063
1,0.092346,4808,444
2,0.094542,2052,194
3,0.081818,330,27
4,0.097561,41,4
5,0.000000,9,0


**Вывод: вероятность появления задолженности по кредиту меньше у заёмщиков не имеющих детей**

При этом, количество детей существенно не влияет на вероятность образования задолженности. Отсутствие выявленной взаимосвязи между количеством детей и вероятностью просрочки может объясняться разным количеством выборки ( с 1 ребёнком - 4808 заёмщиков, с 2 детьми - 2052 заёмщика, 3 ребенка - 330 заёмщиков, 4 ребенка - 41 заёмщик и 5 детей всего 9 заёмщиков)   

#### Определим существует ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
# создадим сводную таблицу по колонке  
data_pivot_married = data.pivot_table(index=['family_status'], values=['debt'], aggfunc = ['mean', 'count', 'sum']) 
data_pivot_married.columns = ['вероятность просрочки','количество клиентов в группе','количество просрочек в группе']
display(data_pivot_married)

,вероятность просрочки,количество клиентов в группе,количество просрочек в группе
family_status,,,
Не женат / не замужем,0.097639,2796,273
в разводе,0.070648,1189,84
вдовец / вдова,0.066246,951,63
гражданский брак,0.093130,4134,385
женат / замужем,0.075606,12261,927


**Вывод: самая низкая вероятность появления задолженности по кредиту у заёмщиков, имеющих статус 'вдовец / вдова' и 'в разводе'** 



#### Определим существует ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
# Установим возможную зависимось между уровнем дохода и возвратом кредита в срок путем группировки по категории доходов (столбец 'total_income_category')
# Иттерируем по уникальным значениям 'total_income_category'. Применим сокращенную логическую индексацию. Метод .sum() т.к. значениями столбца 'debt' могут быть только числа 1 или 0 в зависимости от наличия или отсутствия задолженности
for category in data['total_income_category'].unique():
    category_count = data[data['total_income_category'] == category]['total_income_category'].count() #Количество человек имеющих категорию доходов category
    category_debt = data[(data['total_income_category'] == category) & (data['debt'] != 0)]['debt'].sum() #Количество человек имеющих категорию доходов category и имеющих просрочку
    print(f'Риск задолженности для категории {category} составляет, {category_debt/category_count:.1%}')

#Создадим сводную таблицу 
data_pivot_category_incom = data.pivot_table(index=['total_income_category'], values=['debt'], aggfunc = ['mean', 'count', 'sum']) 
data_pivot_category_incom.columns = ['вероятность просрочки','количество клиентов в группе','количество просрочек в группе']
display(data_pivot_category_incom)    


Риск задолженности для категории B составляет, 7.1%
Риск задолженности для категории C составляет, 8.5%
Риск задолженности для категории D составляет, 6.0%
Риск задолженности для категории E составляет, 9.1%
Риск задолженности для категории A составляет, 8.0%


,вероятность просрочки,количество клиентов в группе,количество просрочек в группе
total_income_category,,,
A,0.080000,25,2
B,0.070602,5014,354
C,0.084982,15921,1353
D,0.060172,349,21
E,0.090909,22,2


**Вывод: Вероятность появления задолженности по кредитам у заёмщиков категории "С" больше чем у заёмщиков категории "В". В связи с малой выборкой заёмщиков с категориями "A", "E" и "D" сделать достоверные выводы о наличии зависимости между уровнем дохода и возвратом кредита в срок по данным категориям не представляется возможным** 

#### Определим как разные цели кредита влияют на его возврат в срок?

In [26]:
# Установим возможную зависимось между целями кредитования и возвратом кредита в срок путем группировки по категории дохода заёмщиков
# Применим метод .sum() т.к. значениями столбца 'debt' могут быть числа 1 или 0 в зависимости от наличия или отсутствия задолженности
for purpose_category in data['purpose_category'].unique():
    category_count = data[data['purpose_category'] == purpose_category]['purpose_category'].count() #Количество человек имеющих категорию доходов category
    category_debt = data[(data['purpose_category'] == purpose_category) & (data['debt'] != 0)]['debt'].sum() #Количество человек имеющих категорию доходов category и имеющих просрочку
    print(f'Риск для категории: {purpose_category} составляет, {category_debt/category_count:.1%}')

#Создадим сводную таблицу 
data_pivot_purpose_category = data.pivot_table(index=['purpose_category'], values=['debt'], aggfunc = ['mean', 'count', 'sum']) 
data_pivot_purpose_category.columns = ['вероятность просрочки','количество клиентов в группе','количество просрочек в группе']
display(data_pivot_purpose_category)    


Риск для категории: операции с недвижимостью составляет, 7.3%
Риск для категории: операции с автомобилем составляет, 9.3%
Риск для категории: получение образования составляет, 9.3%
Риск для категории: проведение свадьбы составляет, 7.9%


,вероятность просрочки,количество клиентов в группе,количество просрочек в группе
purpose_category,,,
операции с автомобилем,0.093480,4279,400
операции с недвижимостью,0.072551,10751,780
получение образования,0.092528,3988,369
проведение свадьбы,0.079118,2313,183


**Вывод: более высокий риск появления задолженности для кредитов, связанных с покупкой автотранспорта и с получением образования**

Но в целом, разница между вероятностью появления просрочки для разных целей не велика и может быть связана с недостаточным количеством данных:
1. Риск для категории: операции с недвижимостью составляет, 7.3%
2. Риск для категории: операции с автомобилем составляет, 9.3%
3. Риск для категории: получение образования составляет, 9.3%
4. Риск для категории: проведение свадьбы составляет, 7.9%

### Общий вывод

С помощью Python и библиотеки pandas было проведено исследование надёжности заёмщиков. Используя при анализе данных методы Python и библиотеки pandas были сделаны следующие выводы:
1. вероятность появления задолженности по кредиту больше у заёмщиков имеющих детей;
2. самая низкая вероятность появления задолженности по кредиту у заёмщиков, имеющих статус 'вдовец / вдова' и 'в разводе', а самая высокая вероятность у заёмщиков с категорией 'гражданский брак';
3. вероятность появления задолженности по кредитам у заёмщиков категории "С" больше чем у заёмщиков категории "В". 

*Выводы, которые нуждаются в дополнительной проверке и могут быть сделаны в данном исследовании из-за различного количества данных по категориям заёмщиков:*
1. более высокий риск появления задолженности по кредитам у заёмщиков, имеющих категорию дохода «Е» (0–30000) и «С» (50001–200000)
2. более высокий риск появления задолженности возникает для кредитов, связанных с покупкой автотранспорта и с получением образования.